<a href="https://colab.research.google.com/github/jonathanyin12/Food.AI/blob/master/FoodAI_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Food.AI Training Notebook**
Easy calorie tracking using real-time object detection on Android. 

**[GitHub](https://github.com/jonathanyin12/Food.AI)**

### **Instructions**:
* Prior to starting, create a directory in Google Drive called *food_detection*. Add the **[training dataset](https://drive.google.com/file/d/11WC6XPp4kHGN1vEzl_ZRnFla99pxIs33/view)** and **[label_map.pbtxt](https://github.com/jonathanyin12/Food.AI/blob/master/label_map.pbtxt)** to *food_detection*.

* Make sure runtime is set to use GPU acceleration

* After installing the correct version of NumPy, restart the runtime and run all the cells after.




In [1]:
# Checking gpu
!nvidia-smi

Sun Dec 27 20:30:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 398.35                 Driver Version: 398.35                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1050   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   33C    P0    N/A /  N/A |     75MiB /  4096MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### **Installing Dependencies**

In [2]:
# IMPORTANT: make sure to restart runtime after running this cell
!pip install numpy==1.17.4

  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 2.4.0 requires numpy~=1.19.2, but you'll have numpy 1.17.4 which is incompatible.


In [3]:
%tensorflow_version 1.x

!git clone --quiet https://github.com/tensorflow/models.git # download training tools

!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz # download model
!tar xvzf ssd_mobilenet_v2_coco_2018_03_29.tar.gz
 
!git clone --quiet https://github.com/zamblauskas/oidv4-toolkit-tfrecord-generator.git # download tfrecord generator tool

!apt-get install -qq protobuf-compiler python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive pycocotools

UsageError: Line magic function `%tensorflow_version` not found.


In [ ]:
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
!python setup.py -q build
!python setup.py -q install

import os
os.environ['PYTHONPATH'] += '/content/models/research/:/content/models/research/slim/'

%cd /content/models/research/slim
!pip install -e .

%cd /content/models/research
!python object_detection/builders/model_builder_test.py

### **Load food dataset and label map from Drive**

In [ ]:
%cd /content/

from google.colab import drive
from zipfile import ZipFile
from shutil import copyfile

drive.mount('/content/gdrive')
copyfile('/content/gdrive/My Drive/food_detection/label_map.pbtxt', 'label_map.pbtxt')

data_path='/content/gdrive/My Drive/food_detection/OIDv4_ToolKit.zip'
with ZipFile(data_path, 'r') as zipObj:
    zipObj.extractall()

### **Convert dataset into TFRecords format**

In [ ]:
!python oidv4-toolkit-tfrecord-generator/generate-tfrecord.py \
    --classes_file=OIDv4_ToolKit/classes.txt \
    --class_descriptions_file=OIDv4_ToolKit/OID/csv_folder/class-descriptions-boxable.csv \
    --annotations_file=OIDv4_ToolKit/OID/csv_folder/train-annotations-bbox.csv \
    --images_dir=OIDv4_ToolKit/OID/Dataset/train \
    --output_file=train.record

!python oidv4-toolkit-tfrecord-generator/generate-tfrecord.py \
    --classes_file=OIDv4_ToolKit/classes.txt \
    --class_descriptions_file=OIDv4_ToolKit/OID/csv_folder/class-descriptions-boxable.csv \
    --annotations_file=OIDv4_ToolKit/OID/csv_folder/validation-annotations-bbox.csv \
    --images_dir=OIDv4_ToolKit/OID/Dataset/validation \
    --output_file=val.record

### **Edit model configuration files**

In [ ]:
import re

filename = '/content/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config'
with open(filename) as f:
    s = f.read()
with open(filename, 'w') as f:
    s = re.sub('90', '15', s) # change number of classes from 90 to 15
    s = re.sub('PATH_TO_BE_CONFIGURED/model.ckpt', '/content/ssd_mobilenet_v2_coco_2018_03_29/model.ckpt', s) # pass location of model checkpoint
    s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_train.record-\?\?\?\?\?-of-00100', '/content/train.record', s) # pass location of train tfrecod
    s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_val.record-\?\?\?\?\?-of-00010', '/content/val.record', s) # pass location of validation tfrecord
    s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt', '/content/label_map.pbtxt', s) # pass location of label map

    f.write(s)

### **Train model**

In [ ]:
!python models/research/object_detection/model_main.py \
    --pipeline_config_path=/content/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config \
    --model_dir=/content/gdrive/My\ Drive/food_detection/model_checkpoints

### **Export model to frozen TensorFlow Lite graph**

In [ ]:
lst = os.listdir('/content/gdrive/My Drive/food_detection/model_checkpoints')
lf = filter(lambda k: 'model.ckpt-' in k, lst)
last_model = sorted(lf)[-1].replace('.meta', '')

!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path=/content/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config \
    --trained_checkpoint_prefix=/content/gdrive/My\ Drive/food_detection/model_checkpoints/$last_model \
    --output_directory=/content/gdrive/My\ Drive/food_detection/model_checkpoints/tflite_model \
    --add_postprocessing_op=true

### **Convert the frozen graph to a TFLite model**

In [ ]:
%cd /content/gdrive/My\ Drive/food_detection/model_checkpoints/tflite_model

!tflite_convert \
    --output_file=food_detect.tflite \
    --graph_def_file=tflite_graph.pb \
    --input_arrays=normalized_input_image_tensor \
    --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'\
    --input_shape=1,300,300,3 \
    --allow_custom_ops